# 1.处理link_info和link_top两个数据集
1. 对于link_info，将每一条路的长度和宽度都标准化到0-1区间；
2. 对于link_top，统计每一条路的入度和出度，并将其标准化到0-1区间；

In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb
from sklearn.utils import shuffle  

C:\ProgramData\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# 读取link_info数据集
link_info = pd.read_csv('./data/gy_contest_link_info.txt', sep = ';')
link_info.head()

,link_ID,length,width,link_class
0,4377906289869500514,57,3,1
1,4377906284594800514,247,9,1
2,4377906289425800514,194,3,1
3,4377906284525800514,839,3,1
4,4377906284422600514,55,12,1


In [3]:
# 利用sklearn中的preprocessing模块的MinMaxScaler函数将长度和宽度进行标准化
min_max_scaler = preprocessing.MinMaxScaler()
link_info['length_std'] = min_max_scaler.fit_transform(link_info['length'])
link_info['width_std'] = min_max_scaler.fit_transform(link_info['width'])

C:\ProgramData\Anaconda2\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)
C:\ProgramData\Anaconda2\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\ProgramData\Anaconda2\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\ProgramData\Anaconda2\lib\site-packages\s

In [4]:
link_info = link_info[['link_ID', 'length_std', 'width_std']]
link_info.head()

,link_ID,length_std,width_std
0,4377906289869500514,0.062350,0.00
1,4377906284594800514,0.290168,0.50
2,4377906289425800514,0.226619,0.00
3,4377906284525800514,1.000000,0.00
4,4377906284422600514,0.059952,0.75


In [5]:
# 读取link_top数据集
link_top = pd.read_csv('./data/gy_contest_link_top.txt', sep=';')
link_top.head()

,link_ID,in_links,out_links
0,4377906289869500514,4377906285525800514,4377906281969500514
1,4377906284594800514,4377906284514600514,4377906285594800514
2,4377906289425800514,NaN,4377906284653600514
3,4377906284525800514,4377906281234600514,4377906280334600514
4,4377906284422600514,3377906289434510514#4377906287959500514,4377906283422600514


In [6]:
# 遍历link_top的每一行，分别计算in_links和out_links的数目
# 将in_links_num和out_links_num加到link_top表中
in_links_num = []
out_links_num = []
for index, row in link_top.iterrows():
    in_links = row[1]
    out_links = row[2]
    if in_links is not np.nan:
        in_links = in_links.strip().split('#')
        in_links_num.append(len(in_links))
    else:
        in_links_num.append(0)
    if out_links is not np.nan:
        out_links = out_links.strip().split('#')
        out_links_num.append(len(out_links))
    else:
        out_links_num.append(0)
link_top['in_links_num'] = in_links_num
link_top['out_links_num'] = out_links_num 

In [7]:
# 利用sklearn中的preprocessing模块的MinMaxScaler函数将长度和宽度进行标准化
link_top['in_links_num_std'] = min_max_scaler.fit_transform(link_top['in_links_num'])
link_top['out_links_num_std'] = min_max_scaler.fit_transform(link_top['out_links_num'])

C:\ProgramData\Anaconda2\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\ProgramData\Anaconda2\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\ProgramData\Anaconda2\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single fea

In [8]:
link_top = link_top[['link_ID', 'in_links_num_std', 'out_links_num_std']]
link_top.head()

,link_ID,in_links_num_std,out_links_num_std
0,4377906289869500514,0.25,0.25
1,4377906284594800514,0.25,0.25
2,4377906289425800514,0.00,0.25
3,4377906284525800514,0.25,0.25
4,4377906284422600514,0.50,0.25


In [9]:
# 将link_info和link_top表合并成link_info_all
link_info_all = pd.merge(link_info, link_top, on='link_ID', how='left')
link_info_all.head()

,link_ID,length_std,width_std,in_links_num_std,out_links_num_std
0,4377906289869500514,0.062350,0.00,0.25,0.25
1,4377906284594800514,0.290168,0.50,0.25,0.25
2,4377906289425800514,0.226619,0.00,0.00,0.25
3,4377906284525800514,1.000000,0.00,0.25,0.25
4,4377906284422600514,0.059952,0.75,0.50,0.25


# 2. 对历史数据集travel_time进行特征提取

In [10]:
## 读取traveltime数据
link_traveling_time = pd.read_table('.\data\quaterfinal_gy_cmp_training_traveltime.txt', sep = ';')
link_traveling_time = link_traveling_time[link_traveling_time['date'] > '2016-12-31']
link_traveling_time['date'] = pd.to_datetime(link_traveling_time['date'])
link_traveling_time = link_traveling_time.sort_values(by=['link_ID','date', 'time_interval'])
# 提取出开始时间和结束时间
link_traveling_time['time_interval_begin'] = pd.to_datetime(link_traveling_time.time_interval.map(lambda x : x.strip().split(',')[0][1:]))
link_traveling_time['time_interval_end'] = pd.to_datetime(link_traveling_time.time_interval.map(lambda x : x.strip().split(',')[1][:-1]))
# 提取出时钟和分钟信息
link_traveling_time['time_interval_hour'] = link_traveling_time['time_interval_begin'].map(lambda x : x.strftime('%H'))
link_traveling_time['time_interval_minutes'] = link_traveling_time['time_interval_begin'].map(lambda x : x.strftime('%M'))
link_traveling_time.head()

,link_ID,date,time_interval,travel_time,time_interval_begin,time_interval_end,time_interval_hour,time_interval_minutes
5281666,3377906280028510514,2017-04-01,"[2017-04-01 00:00:00,2017-04-01 00:02:00)",5.4,2017-04-01 00:00:00,2017-04-01 00:02:00,00,00
5293738,3377906280028510514,2017-04-01,"[2017-04-01 00:02:00,2017-04-01 00:04:00)",5.4,2017-04-01 00:02:00,2017-04-01 00:04:00,00,02
5263441,3377906280028510514,2017-04-01,"[2017-04-01 00:08:00,2017-04-01 00:10:00)",4.4,2017-04-01 00:08:00,2017-04-01 00:10:00,00,08
5289277,3377906280028510514,2017-04-01,"[2017-04-01 00:10:00,2017-04-01 00:12:00)",4.4,2017-04-01 00:10:00,2017-04-01 00:12:00,00,10
5262917,3377906280028510514,2017-04-01,"[2017-04-01 00:12:00,2017-04-01 00:14:00)",4.4,2017-04-01 00:12:00,2017-04-01 00:14:00,00,12


In [11]:
#定义求众数的函数
from scipy.stats import mode
def mode_function(df):
    counts = mode(df)
    return counts[0][0]

In [12]:
# 统计每一天7点中的均值、中位数、标准差、众数、最大值和最小值
temp1 = link_traveling_time[link_traveling_time.time_interval_hour == '07']
temp1_stats = temp1[['link_ID', 'date', 'time_interval_hour', 'travel_time']].groupby(['link_ID', 'date', 'time_interval_hour']).agg([('mean_last_1hour',np.mean),('median_last_1hour',np.median),('std_last_1hour',np.std),('mode_last_1hour',mode_function),('max_last_1hour',np.max), ('min_last_1hour', np.min)])
temp1_stats.columns = temp1_stats.columns.droplevel(0)
temp1_stats = temp1_stats.reset_index()
temp1_stats = temp1_stats.fillna(0)
temp1_stats.head()

,link_ID,date,time_interval_hour,mean_last_1hour,median_last_1hour,std_last_1hour,mode_last_1hour,max_last_1hour,min_last_1hour
0,3377906280028510514,2017-04-01,07,5.483333,4.75,1.343751,4.5,7.2,4.5
1,3377906280028510514,2017-04-02,07,3.600000,3.60,0.000000,3.6,3.6,3.6
2,3377906280028510514,2017-04-04,07,4.000000,4.00,0.000000,4.0,4.0,4.0
3,3377906280028510514,2017-04-05,07,6.038462,6.00,1.154367,7.0,7.0,3.8
4,3377906280028510514,2017-04-06,07,6.409091,6.70,0.980260,6.7,8.0,5.3


In [13]:
# 统计每天7点40到8点的均值、中位数、标准差、众数、最大值和最小值
temp2 = link_traveling_time[(link_traveling_time.time_interval_hour == '07') & (link_traveling_time.time_interval_minutes >= '40')]
temp2_stats = temp2[['link_ID', 'date', 'time_interval_hour', 'travel_time']].groupby(['link_ID', 'date', 'time_interval_hour']).agg([('mean_last_half_hour',np.mean),('median_last_half_hour',np.median),('std_last_half_hour',np.std),('mode_last_half_hour',mode_function),('max_last_half_hour',np.max), ('min_last_half_hour', np.min)])
temp2_stats.columns = temp2_stats.columns.droplevel(0)
temp2_stats = temp2_stats.reset_index()
temp2_stats = temp2_stats.fillna(0)
temp2_stats.head()

,link_ID,date,time_interval_hour,mean_last_half_hour,median_last_half_hour,std_last_half_hour,mode_last_half_hour,max_last_half_hour,min_last_half_hour
0,3377906280028510514,2017-04-05,07,6.520000,7.0,0.657267,7.0,7.0,5.8
1,3377906280028510514,2017-04-06,07,7.133333,6.7,0.671317,6.7,8.0,6.7
2,3377906280028510514,2017-04-08,07,5.333333,5.5,0.258199,5.5,5.5,5.0
3,3377906280028510514,2017-04-10,07,6.000000,6.0,0.000000,6.0,6.0,6.0
4,3377906280028510514,2017-04-11,07,6.850000,7.0,1.618332,7.0,8.5,4.3


In [14]:
# 统计每一天14点中的均值、中位数、标准差、众数、最大值和最小值
temp3 = link_traveling_time[link_traveling_time.time_interval_hour == '14']
temp3_stats = temp3[['link_ID', 'date', 'time_interval_hour', 'travel_time']].groupby(['link_ID', 'date', 'time_interval_hour']).agg([('mean_last_1hour',np.mean),('median_last_1hour',np.median),('std_last_1hour',np.std),('mode_last_1hour',mode_function),('max_last_1hour',np.max), ('min_last_1hour', np.min)])
temp3_stats.columns = temp3_stats.columns.droplevel(0)
temp3_stats = temp3_stats.reset_index()
temp3_stats = temp3_stats.fillna(0)
temp3_stats.head()

,link_ID,date,time_interval_hour,mean_last_1hour,median_last_1hour,std_last_1hour,mode_last_1hour,max_last_1hour,min_last_1hour
0,3377906280028510514,2017-04-01,14,9.847826,6.5,7.128233,4.9,27.9,4.2
1,3377906280028510514,2017-04-02,14,7.181818,6.9,1.250766,7.0,10.7,5.0
2,3377906280028510514,2017-04-03,14,6.024000,6.2,1.881551,6.2,12.9,3.8
3,3377906280028510514,2017-04-04,14,7.082609,7.5,1.684866,4.8,9.9,4.8
4,3377906280028510514,2017-04-05,14,6.183333,5.8,2.340249,7.0,11.5,2.9


In [15]:
# 统计每天14点40到8点的均值、中位数、标准差、众数、最大值和最小值
temp4 = link_traveling_time[(link_traveling_time.time_interval_hour == '14') & (link_traveling_time.time_interval_minutes >= '40')]
temp4_stats = temp4[['link_ID', 'date', 'time_interval_hour', 'travel_time']].groupby(['link_ID', 'date', 'time_interval_hour']).agg([('mean_last_half_hour',np.mean),('median_last_half_hour',np.median),('std_last_half_hour',np.std),('mode_last_half_hour',mode_function),('max_last_half_hour',np.max), ('min_last_half_hour', np.min)])
temp4_stats.columns = temp4_stats.columns.droplevel(0)
temp4_stats = temp4_stats.reset_index()
temp4_stats = temp4_stats.fillna(0)
temp4_stats.head()

,link_ID,date,time_interval_hour,mean_last_half_hour,median_last_half_hour,std_last_half_hour,mode_last_half_hour,max_last_half_hour,min_last_half_hour
0,3377906280028510514,2017-04-01,14,4.76,4.90,0.313050,4.9,4.9,4.2
1,3377906280028510514,2017-04-02,14,6.30,6.50,0.589491,6.8,6.8,5.0
2,3377906280028510514,2017-04-03,14,6.21,6.20,1.062962,6.2,7.4,4.6
3,3377906280028510514,2017-04-04,14,7.40,7.80,0.952190,8.0,8.0,6.0
4,3377906280028510514,2017-04-05,14,4.95,4.95,1.364022,4.5,7.0,2.9


In [16]:
# 统计每一天17点中的均值、中位数、标准差、众数、最大值和最小值
temp5 = link_traveling_time[link_traveling_time.time_interval_hour == '17']
temp5_stats = temp5[['link_ID', 'date', 'time_interval_hour', 'travel_time']].groupby(['link_ID', 'date', 'time_interval_hour']).agg([('mean_last_1hour',np.mean),('median_last_1hour',np.median),('std_last_1hour',np.std),('mode_last_1hour',mode_function),('max_last_1hour',np.max), ('min_last_1hour', np.min)])
temp5_stats.columns = temp5_stats.columns.droplevel(0)
temp5_stats = temp5_stats.reset_index()
temp5_stats = temp5_stats.fillna(0)
temp5_stats.head()

,link_ID,date,time_interval_hour,mean_last_1hour,median_last_1hour,std_last_1hour,mode_last_1hour,max_last_1hour,min_last_1hour
0,3377906280028510514,2017-04-01,17,9.140909,6.90,6.429001,6.9,36.1,4.6
1,3377906280028510514,2017-04-02,17,6.544000,6.90,0.975141,6.9,8.0,4.4
2,3377906280028510514,2017-04-03,17,6.156522,6.30,1.197280,5.3,9.1,4.4
3,3377906280028510514,2017-04-04,17,6.092857,6.15,1.187880,6.5,7.9,3.6
4,3377906280028510514,2017-04-05,17,6.400000,6.20,1.920069,5.2,9.4,2.8


In [17]:
# 统计每天17点40到8点的均值、中位数、标准差、众数、最大值和最小值
temp6 = link_traveling_time[(link_traveling_time.time_interval_hour == '17') & (link_traveling_time.time_interval_minutes >= '40')]
temp6_stats = temp6[['link_ID', 'date', 'time_interval_hour', 'travel_time']].groupby(['link_ID', 'date', 'time_interval_hour']).agg([('mean_last_half_hour',np.mean),('median_last_half_hour',np.median),('std_last_half_hour',np.std),('mode_last_half_hour',mode_function),('max_last_half_hour',np.max), ('min_last_half_hour', np.min)])
temp6_stats.columns = temp6_stats.columns.droplevel(0)
temp6_stats = temp6_stats.reset_index()
temp6_stats = temp6_stats.fillna(0)
temp6_stats.head()

,link_ID,date,time_interval_hour,mean_last_half_hour,median_last_half_hour,std_last_half_hour,mode_last_half_hour,max_last_half_hour,min_last_half_hour
0,3377906280028510514,2017-04-01,17,7.040000,6.90,0.863069,6.9,9.4,6.2
1,3377906280028510514,2017-04-02,17,7.160000,7.20,0.573876,7.1,8.0,6.3
2,3377906280028510514,2017-04-03,17,5.187500,5.00,0.797205,5.0,7.0,4.4
3,3377906280028510514,2017-04-04,17,6.420000,6.15,0.991968,5.2,7.9,5.2
4,3377906280028510514,2017-04-05,17,5.983333,6.40,2.858263,8.0,9.4,2.8


In [18]:
## 将两个统计数据整合到一张表中
travel_time_stats_7h = pd.merge(temp1_stats, temp2_stats, on = ['link_ID', 'date', 'time_interval_hour'], how='left')
travel_time_stats_14h = pd.merge(temp3_stats, temp4_stats, on = ['link_ID', 'date', 'time_interval_hour'], how='left')
travel_time_stats_17h = pd.merge(temp5_stats, temp6_stats, on = ['link_ID', 'date', 'time_interval_hour'], how='left')

In [19]:
# 填充缺失值
travel_time_stats_7h = travel_time_stats_7h.fillna({'mean_last_half_hour':travel_time_stats_7h['mean_last_1hour'],
                                                    'median_last_half_hour': travel_time_stats_7h['median_last_1hour'],
                                                    'std_last_half_hour': travel_time_stats_7h['std_last_1hour'],
                                                    'mode_last_half_hour': travel_time_stats_7h['mode_last_1hour'],
                                                    'max_last_half_hour': travel_time_stats_7h['max_last_1hour'],
                                                    'min_last_half_hour': travel_time_stats_7h['min_last_1hour']})
travel_time_stats_14h = travel_time_stats_14h.fillna({'mean_last_half_hour':travel_time_stats_14h['mean_last_1hour'],
                                                    'median_last_half_hour': travel_time_stats_14h['median_last_1hour'],
                                                    'std_last_half_hour': travel_time_stats_14h['std_last_1hour'],
                                                    'mode_last_half_hour': travel_time_stats_14h['mode_last_1hour'],
                                                    'max_last_half_hour': travel_time_stats_14h['max_last_1hour'],
                                                    'min_last_half_hour': travel_time_stats_14h['min_last_1hour']})
travel_time_stats_17h = travel_time_stats_17h.fillna({'mean_last_half_hour':travel_time_stats_17h['mean_last_1hour'],
                                                    'median_last_half_hour': travel_time_stats_17h['median_last_1hour'],
                                                    'std_last_half_hour': travel_time_stats_17h['std_last_1hour'],
                                                    'mode_last_half_hour': travel_time_stats_17h['mode_last_1hour'],
                                                    'max_last_half_hour': travel_time_stats_17h['max_last_1hour'],
                                                    'min_last_half_hour': travel_time_stats_17h['min_last_1hour']})

In [20]:
# 由于7点的统计数据需要整合到8点的训练数据中，因此将time_interval_hoour的值加1（改成08）
travel_time_stats_7h['time_interval_hour'] = '08'
travel_time_stats_14h['time_interval_hour'] = '15'
travel_time_stats_17h['time_interval_hour'] = '18'

In [21]:
# 增加一列表明是上午、下午和傍晚
travel_time_stats_7h['time_stage1'] = 1
travel_time_stats_7h['time_stage2'] = 0
travel_time_stats_7h['time_stage3'] = 0

travel_time_stats_14h['time_stage1'] = 0
travel_time_stats_14h['time_stage2'] = 1
travel_time_stats_14h['time_stage3'] = 0

travel_time_stats_17h['time_stage1'] = 0
travel_time_stats_17h['time_stage2'] = 0
travel_time_stats_17h['time_stage3'] = 1

In [22]:
## 连接整合出8点、15点和18点的前一小时统计数据
travel_time_stats = pd.concat([travel_time_stats_7h, travel_time_stats_14h, travel_time_stats_17h])
travel_time_stats.head()

,link_ID,date,time_interval_hour,mean_last_1hour,median_last_1hour,std_last_1hour,mode_last_1hour,max_last_1hour,min_last_1hour,mean_last_half_hour,median_last_half_hour,std_last_half_hour,mode_last_half_hour,max_last_half_hour,min_last_half_hour,time_stage1,time_stage2,time_stage3
0,3377906280028510514,2017-04-01,08,5.483333,4.75,1.343751,4.5,7.2,4.5,5.483333,4.75,1.343751,4.5,7.2,4.5,1,0,0
1,3377906280028510514,2017-04-02,08,3.600000,3.60,0.000000,3.6,3.6,3.6,3.600000,3.60,0.000000,3.6,3.6,3.6,1,0,0
2,3377906280028510514,2017-04-04,08,4.000000,4.00,0.000000,4.0,4.0,4.0,4.000000,4.00,0.000000,4.0,4.0,4.0,1,0,0
3,3377906280028510514,2017-04-05,08,6.038462,6.00,1.154367,7.0,7.0,3.8,6.520000,7.00,0.657267,7.0,7.0,5.8,1,0,0
4,3377906280028510514,2017-04-06,08,6.409091,6.70,0.980260,6.7,8.0,5.3,7.133333,6.70,0.671317,6.7,8.0,6.7,1,0,0


In [23]:
len(travel_time_stats)

47394

# 3. 生成训练数据集

In [24]:
travel_time_8h = link_traveling_time[link_traveling_time['time_interval_hour'] == '08']
travel_time_15h = link_traveling_time[link_traveling_time['time_interval_hour'] == '15']
travel_time_18h = link_traveling_time[link_traveling_time['time_interval_hour'] == '18']

In [25]:
## 根据time_interval_minutes计算该时间段距离8点的间隔数
travel_time_8h['interval_num'] = travel_time_8h['time_interval_minutes'].astype(int) / 2
## 根据time_interval_minutes计算该时间段距离15点的间隔数
travel_time_15h['interval_num'] = travel_time_15h['time_interval_minutes'].astype(int) / 2
## 根据time_interval_minutes计算该时间段距离18点的间隔数
travel_time_18h['interval_num'] = travel_time_18h['time_interval_minutes'].astype(int) / 2

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org

In [26]:
travel_time_train = pd.concat([travel_time_8h, travel_time_15h, travel_time_18h])
travel_time_train.head()

,link_ID,date,time_interval,travel_time,time_interval_begin,time_interval_end,time_interval_hour,time_interval_minutes,interval_num
5222026,3377906280028510514,2017-04-01,"[2017-04-01 08:08:00,2017-04-01 08:10:00)",17.3,2017-04-01 08:08:00,2017-04-01 08:10:00,08,08,4.0
5239651,3377906280028510514,2017-04-01,"[2017-04-01 08:10:00,2017-04-01 08:12:00)",17.3,2017-04-01 08:10:00,2017-04-01 08:12:00,08,10,5.0
5259520,3377906280028510514,2017-04-01,"[2017-04-01 08:12:00,2017-04-01 08:14:00)",17.3,2017-04-01 08:12:00,2017-04-01 08:14:00,08,12,6.0
5290134,3377906280028510514,2017-04-01,"[2017-04-01 08:16:00,2017-04-01 08:18:00)",5.7,2017-04-01 08:16:00,2017-04-01 08:18:00,08,16,8.0
5216769,3377906280028510514,2017-04-01,"[2017-04-01 08:18:00,2017-04-01 08:20:00)",5.7,2017-04-01 08:18:00,2017-04-01 08:20:00,08,18,9.0


In [27]:
len(travel_time_train)

1029603

In [28]:
#将统计数据整合到训练数据集中
travel_time_train = pd.merge(travel_time_train, travel_time_stats, on=['link_ID', 'date', 'time_interval_hour'], how='left')
travel_time_train.head()

,link_ID,date,time_interval,travel_time,time_interval_begin,time_interval_end,time_interval_hour,time_interval_minutes,interval_num,mean_last_1hour,...,min_last_1hour,mean_last_half_hour,median_last_half_hour,std_last_half_hour,mode_last_half_hour,max_last_half_hour,min_last_half_hour,time_stage1,time_stage2,time_stage3
0,3377906280028510514,2017-04-01,"[2017-04-01 08:08:00,2017-04-01 08:10:00)",17.3,2017-04-01 08:08:00,2017-04-01 08:10:00,08,08,4.0,5.483333,...,4.5,5.483333,4.75,1.343751,4.5,7.2,4.5,1.0,0.0,0.0
1,3377906280028510514,2017-04-01,"[2017-04-01 08:10:00,2017-04-01 08:12:00)",17.3,2017-04-01 08:10:00,2017-04-01 08:12:00,08,10,5.0,5.483333,...,4.5,5.483333,4.75,1.343751,4.5,7.2,4.5,1.0,0.0,0.0
2,3377906280028510514,2017-04-01,"[2017-04-01 08:12:00,2017-04-01 08:14:00)",17.3,2017-04-01 08:12:00,2017-04-01 08:14:00,08,12,6.0,5.483333,...,4.5,5.483333,4.75,1.343751,4.5,7.2,4.5,1.0,0.0,0.0
3,3377906280028510514,2017-04-01,"[2017-04-01 08:16:00,2017-04-01 08:18:00)",5.7,2017-04-01 08:16:00,2017-04-01 08:18:00,08,16,8.0,5.483333,...,4.5,5.483333,4.75,1.343751,4.5,7.2,4.5,1.0,0.0,0.0
4,3377906280028510514,2017-04-01,"[2017-04-01 08:18:00,2017-04-01 08:20:00)",5.7,2017-04-01 08:18:00,2017-04-01 08:20:00,08,18,9.0,5.483333,...,4.5,5.483333,4.75,1.343751,4.5,7.2,4.5,1.0,0.0,0.0


In [29]:
# 生成时间间隔的one_hot编码表
interval_num = travel_time_train['interval_num']
interval_num = [[i] for i in interval_num]
enc = preprocessing.OneHotEncoder()
enc.fit(interval_num)
interval_num_onehot = enc.transform(interval_num).toarray()
columns = ['interval_num_%d'% i for i in range(1, 31)]
one_hot = pd.DataFrame(interval_num_onehot, columns = columns)

for i in range(len(columns)):
    travel_time_train[columns[i]] = one_hot[columns[i]]

In [30]:
##直接将时间间隔树归一化
#travel_time_train['interval_num'] = min_max_scaler.fit_transform(travel_time_train['interval_num'])

In [31]:
#增加星期信息
travel_time_train['dow'] = travel_time_train['date'].apply(lambda x: x.dayofweek)

In [32]:
# 生成时间间隔的one_hot编码表
dayOfWeek = travel_time_train['dow']
dayOfWeek_num = [[i] for i in dayOfWeek]
enc = preprocessing.OneHotEncoder()
enc.fit(dayOfWeek_num)
dayOfWeek_onehot = enc.transform(dayOfWeek_num).toarray()
columns = ['dow_%d'% i for i in range(1, 8)]
one_hot = pd.DataFrame(dayOfWeek_onehot, columns = columns)

for i in range(len(columns)):
    travel_time_train[columns[i]] = one_hot[columns[i]]

In [33]:
travel_time_train.head()

,link_ID,date,time_interval,travel_time,time_interval_begin,time_interval_end,time_interval_hour,time_interval_minutes,interval_num,mean_last_1hour,...,interval_num_29,interval_num_30,dow,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6,dow_7
0,3377906280028510514,2017-04-01,"[2017-04-01 08:08:00,2017-04-01 08:10:00)",17.3,2017-04-01 08:08:00,2017-04-01 08:10:00,08,08,4.0,5.483333,...,0.0,0.0,5,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,3377906280028510514,2017-04-01,"[2017-04-01 08:10:00,2017-04-01 08:12:00)",17.3,2017-04-01 08:10:00,2017-04-01 08:12:00,08,10,5.0,5.483333,...,0.0,0.0,5,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,3377906280028510514,2017-04-01,"[2017-04-01 08:12:00,2017-04-01 08:14:00)",17.3,2017-04-01 08:12:00,2017-04-01 08:14:00,08,12,6.0,5.483333,...,0.0,0.0,5,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,3377906280028510514,2017-04-01,"[2017-04-01 08:16:00,2017-04-01 08:18:00)",5.7,2017-04-01 08:16:00,2017-04-01 08:18:00,08,16,8.0,5.483333,...,0.0,0.0,5,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,3377906280028510514,2017-04-01,"[2017-04-01 08:18:00,2017-04-01 08:20:00)",5.7,2017-04-01 08:18:00,2017-04-01 08:20:00,08,18,9.0,5.483333,...,0.0,0.0,5,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [34]:
del travel_time_train['dow']

In [35]:
## 整合link_info的信息
travel_time_train = pd.merge(travel_time_train, link_info_all, on='link_ID', how='left')

In [36]:
len(travel_time_train)

1029603

In [37]:
travel_time_train = travel_time_train.dropna()

In [38]:
travel_time_train.isnull().any()

link_ID                  False
date                     False
time_interval            False
travel_time              False
time_interval_begin      False
time_interval_end        False
time_interval_hour       False
time_interval_minutes    False
interval_num             False
mean_last_1hour          False
median_last_1hour        False
std_last_1hour           False
mode_last_1hour          False
max_last_1hour           False
min_last_1hour           False
mean_last_half_hour      False
median_last_half_hour    False
std_last_half_hour       False
mode_last_half_hour      False
max_last_half_hour       False
min_last_half_hour       False
time_stage1              False
time_stage2              False
time_stage3              False
interval_num_1           False
interval_num_2           False
interval_num_3           False
interval_num_4           False
interval_num_5           False
interval_num_6           False
                         ...  
interval_num_12          False
interval

In [39]:
len(travel_time_train)

1029440

In [40]:
#shuffle函数等到最后再用
#travel_time_train_shuffle = shuffle(travel_time_train)

In [41]:
## 将travel_time表的日期后移7天和15天，用于整合到特征中，表示该时刻7天和15天前的数据。
link_traveling_time_shift_7 = link_traveling_time.copy()
link_traveling_time_shift_7['date'] = pd.DataFrame(link_traveling_time_shift_7['date']+pd.Timedelta('7 days'))
link_traveling_time_shift_7.head()

,link_ID,date,time_interval,travel_time,time_interval_begin,time_interval_end,time_interval_hour,time_interval_minutes
5281666,3377906280028510514,2017-04-08,"[2017-04-01 00:00:00,2017-04-01 00:02:00)",5.4,2017-04-01 00:00:00,2017-04-01 00:02:00,00,00
5293738,3377906280028510514,2017-04-08,"[2017-04-01 00:02:00,2017-04-01 00:04:00)",5.4,2017-04-01 00:02:00,2017-04-01 00:04:00,00,02
5263441,3377906280028510514,2017-04-08,"[2017-04-01 00:08:00,2017-04-01 00:10:00)",4.4,2017-04-01 00:08:00,2017-04-01 00:10:00,00,08
5289277,3377906280028510514,2017-04-08,"[2017-04-01 00:10:00,2017-04-01 00:12:00)",4.4,2017-04-01 00:10:00,2017-04-01 00:12:00,00,10
5262917,3377906280028510514,2017-04-08,"[2017-04-01 00:12:00,2017-04-01 00:14:00)",4.4,2017-04-01 00:12:00,2017-04-01 00:14:00,00,12


In [42]:
link_traveling_time_shift_15 = link_traveling_time.copy()
link_traveling_time_shift_15['date'] = pd.DataFrame(link_traveling_time_shift_15['date']+pd.Timedelta('15 days'))
link_traveling_time_shift_15.head()

,link_ID,date,time_interval,travel_time,time_interval_begin,time_interval_end,time_interval_hour,time_interval_minutes
5281666,3377906280028510514,2017-04-16,"[2017-04-01 00:00:00,2017-04-01 00:02:00)",5.4,2017-04-01 00:00:00,2017-04-01 00:02:00,00,00
5293738,3377906280028510514,2017-04-16,"[2017-04-01 00:02:00,2017-04-01 00:04:00)",5.4,2017-04-01 00:02:00,2017-04-01 00:04:00,00,02
5263441,3377906280028510514,2017-04-16,"[2017-04-01 00:08:00,2017-04-01 00:10:00)",4.4,2017-04-01 00:08:00,2017-04-01 00:10:00,00,08
5289277,3377906280028510514,2017-04-16,"[2017-04-01 00:10:00,2017-04-01 00:12:00)",4.4,2017-04-01 00:10:00,2017-04-01 00:12:00,00,10
5262917,3377906280028510514,2017-04-16,"[2017-04-01 00:12:00,2017-04-01 00:14:00)",4.4,2017-04-01 00:12:00,2017-04-01 00:14:00,00,12


In [43]:
# 选取'link_ID', 'date', 'time_interval_hour', 'time_interval_minutes', 'travel_time'
link_traveling_time_shift_7 = link_traveling_time_shift_7[['link_ID', 'date', 'time_interval_hour', 'time_interval_minutes', 'travel_time']]
link_traveling_time_shift_15 = link_traveling_time_shift_15[['link_ID', 'date', 'time_interval_hour', 'time_interval_minutes', 'travel_time']]

In [44]:
link_traveling_time_shift_7['travel_time_7days_before'] = link_traveling_time_shift_7['travel_time']
link_traveling_time_shift_15['travel_time_15days_before'] = link_traveling_time_shift_15['travel_time']
del link_traveling_time_shift_7['travel_time']
del link_traveling_time_shift_15['travel_time']

In [45]:
link_traveling_time_shift_7.head()

,link_ID,date,time_interval_hour,time_interval_minutes,travel_time_7days_before
5281666,3377906280028510514,2017-04-08,00,00,5.4
5293738,3377906280028510514,2017-04-08,00,02,5.4
5263441,3377906280028510514,2017-04-08,00,08,4.4
5289277,3377906280028510514,2017-04-08,00,10,4.4
5262917,3377906280028510514,2017-04-08,00,12,4.4


In [46]:
link_traveling_time_shift_7_before_july = link_traveling_time_shift_7[link_traveling_time_shift_7['date'] < '2017-07-01']
link_traveling_time_shift_15_before_july = link_traveling_time_shift_15[link_traveling_time_shift_15['date'] < '2017-07-01']

In [47]:
travel_time_train = pd.merge(travel_time_train, link_traveling_time_shift_7_before_july, on=['link_ID', 'date', 'time_interval_hour', 'time_interval_minutes'], how='left')
travel_time_train = pd.merge(travel_time_train, link_traveling_time_shift_15_before_july, on=['link_ID', 'date', 'time_interval_hour', 'time_interval_minutes'], how='left')

In [48]:
travel_time_train = travel_time_train.dropna()

In [49]:
len(travel_time_train)

808159

In [50]:
##计算每一条路的所有历史统计数据
temp_stats_all = link_traveling_time[['link_ID', 'time_interval_hour', 'time_interval_minutes', 'travel_time']].groupby(['link_ID', 'time_interval_hour', 'time_interval_minutes']).agg([('mean_all',np.mean),('median_all',np.median),('std_all',np.std),('mode_all',mode_function)])
temp_stats_all.columns = temp_stats_all.columns.droplevel(0)
temp_stats_all = temp_stats_all.reset_index()
temp_stats_all = temp_stats_all.fillna(0)
temp_stats_all.head()

,link_ID,time_interval_hour,time_interval_minutes,mean_all,median_all,std_all,mode_all
0,3377906280028510514,00,00,4.984211,4.85,1.460568,5.0
1,3377906280028510514,00,02,4.969767,4.90,1.388307,5.0
2,3377906280028510514,00,04,4.768571,4.80,1.006967,4.8
3,3377906280028510514,00,06,4.700000,4.75,0.804716,5.2
4,3377906280028510514,00,08,4.750000,4.60,1.085211,4.2


In [51]:
travel_time_train = pd.merge(travel_time_train, temp_stats_all, on=['link_ID', 'time_interval_hour', 'time_interval_minutes'], how='left')

In [52]:
len(travel_time_train)

808159

In [53]:
travel_time_train.isnull().any()

link_ID                      False
date                         False
time_interval                False
travel_time                  False
time_interval_begin          False
time_interval_end            False
time_interval_hour           False
time_interval_minutes        False
interval_num                 False
mean_last_1hour              False
median_last_1hour            False
std_last_1hour               False
mode_last_1hour              False
max_last_1hour               False
min_last_1hour               False
mean_last_half_hour          False
median_last_half_hour        False
std_last_half_hour           False
mode_last_half_hour          False
max_last_half_hour           False
min_last_half_hour           False
time_stage1                  False
time_stage2                  False
time_stage3                  False
interval_num_1               False
interval_num_2               False
interval_num_3               False
interval_num_4               False
interval_num_5      

In [54]:
#将travel time 移到最后一列（这一步最后再做）
travel_time_train['travel_time_tmp'] = travel_time_train['travel_time']
del travel_time_train['travel_time']
travel_time_train['travel_time'] = travel_time_train['travel_time_tmp']
del travel_time_train['travel_time_tmp']

# 4.生成预测数据集

In [98]:
prediction_table = pd.read_csv('./data/semifinal_gy_cmp_testing_template_seg2.txt', sep=';', names = ['link_ID', 'date', 'time_interval', 'travel_time'])
prediction_table.head()

,link_ID,date,time_interval,travel_time
0,4377906289869500514,2017-07-01,"[2017-07-01 08:00:00,2017-07-01 08:02:00)",0
1,4377906289869500514,2017-07-01,"[2017-07-01 08:02:00,2017-07-01 08:04:00)",0
2,4377906289869500514,2017-07-01,"[2017-07-01 08:04:00,2017-07-01 08:06:00)",0
3,4377906289869500514,2017-07-01,"[2017-07-01 08:06:00,2017-07-01 08:08:00)",0
4,4377906289869500514,2017-07-01,"[2017-07-01 08:08:00,2017-07-01 08:10:00)",0


In [99]:
len(prediction_table)

368280

In [100]:
prediction_table['date'] = pd.to_datetime(prediction_table['date'])
prediction_table = prediction_table.sort_values(by=['link_ID','date', 'time_interval'])

prediction_table['time_interval_begin'] = pd.to_datetime(prediction_table.time_interval.map(lambda x : x.strip().split(',')[0][1:]))
prediction_table['time_interval_end'] = pd.to_datetime(prediction_table.time_interval.map(lambda x : x.strip().split(',')[1][:-1]))

prediction_table['time_interval_hour'] = prediction_table['time_interval_begin'].map(lambda x : x.strftime('%H'))
prediction_table['time_interval_minutes'] = prediction_table['time_interval_begin'].map(lambda x : x.strftime('%M'))

prediction_table['interval_num'] = prediction_table['time_interval_minutes'].astype(int) / 2

prediction_table = pd.merge(prediction_table, travel_time_stats, on=['link_ID', 'date', 'time_interval_hour'], how='left')
prediction_table.head()

,link_ID,date,time_interval,travel_time,time_interval_begin,time_interval_end,time_interval_hour,time_interval_minutes,interval_num,mean_last_1hour,...,min_last_1hour,mean_last_half_hour,median_last_half_hour,std_last_half_hour,mode_last_half_hour,max_last_half_hour,min_last_half_hour,time_stage1,time_stage2,time_stage3
0,3377906280028510514,2017-07-01,"[2017-07-01 08:00:00,2017-07-01 08:02:00)",0,2017-07-01 08:00:00,2017-07-01 08:02:00,08,00,0.0,5.742857,...,3.5,5.025,5.25,0.713559,4.0,5.6,4.0,1.0,0.0,0.0
1,3377906280028510514,2017-07-01,"[2017-07-01 08:02:00,2017-07-01 08:04:00)",0,2017-07-01 08:02:00,2017-07-01 08:04:00,08,02,1.0,5.742857,...,3.5,5.025,5.25,0.713559,4.0,5.6,4.0,1.0,0.0,0.0
2,3377906280028510514,2017-07-01,"[2017-07-01 08:04:00,2017-07-01 08:06:00)",0,2017-07-01 08:04:00,2017-07-01 08:06:00,08,04,2.0,5.742857,...,3.5,5.025,5.25,0.713559,4.0,5.6,4.0,1.0,0.0,0.0
3,3377906280028510514,2017-07-01,"[2017-07-01 08:06:00,2017-07-01 08:08:00)",0,2017-07-01 08:06:00,2017-07-01 08:08:00,08,06,3.0,5.742857,...,3.5,5.025,5.25,0.713559,4.0,5.6,4.0,1.0,0.0,0.0
4,3377906280028510514,2017-07-01,"[2017-07-01 08:08:00,2017-07-01 08:10:00)",0,2017-07-01 08:08:00,2017-07-01 08:10:00,08,08,4.0,5.742857,...,3.5,5.025,5.25,0.713559,4.0,5.6,4.0,1.0,0.0,0.0


In [101]:
prediction_table = prediction_table.fillna(method = 'bfill')
prediction_table = prediction_table.fillna(0)

In [102]:
# 生成时间间隔的one_hot编码表
interval_num = prediction_table['interval_num']
interval_num = [[i] for i in interval_num]
enc = preprocessing.OneHotEncoder()
enc.fit(interval_num)
interval_num_onehot = enc.transform(interval_num).toarray()
columns = ['interval_num_%d'% i for i in range(1, 31)]
one_hot = pd.DataFrame(interval_num_onehot, columns = columns)

for i in range(len(columns)):
    prediction_table[columns[i]] = one_hot[columns[i]]

In [103]:
##直接将时间间隔树归一化
#prediction_table['interval_num'] = min_max_scaler.fit_transform(prediction_table['interval_num'])

In [104]:
#增加星期信息
prediction_table['dow'] = prediction_table['date'].apply(lambda x: x.dayofweek)

In [105]:
# 生成时间间隔的one_hot编码表
dayOfWeek = prediction_table['dow']
dayOfWeek_num = [[i] for i in dayOfWeek]
enc = preprocessing.OneHotEncoder()
enc.fit(dayOfWeek_num)
dayOfWeek_onehot = enc.transform(dayOfWeek_num).toarray()
columns = ['dow_%d'% i for i in range(1, 8)]
one_hot = pd.DataFrame(dayOfWeek_onehot, columns = columns)

for i in range(len(columns)):
    prediction_table[columns[i]] = one_hot[columns[i]]

In [106]:
del prediction_table['dow']

In [107]:
prediction_table = pd.merge(prediction_table, link_info_all, on='link_ID', how='left')

In [108]:
len(prediction_table)

368280

In [109]:
#取平移7天和15天的数据
#注意：7天的数据，只能得到7月1日到7月7日的数据
link_traveling_time_shift_7_71_77 = link_traveling_time_shift_7[(link_traveling_time_shift_7['date']>='2017-07-01')&(link_traveling_time_shift_7['date']<='2017-07-07')]
#平移第一次7天
link_traveling_time_shift_7_71_77_reshift1 = link_traveling_time_shift_7_71_77.copy()
link_traveling_time_shift_7_71_77_reshift1['date'] = pd.DataFrame(link_traveling_time_shift_7_71_77_reshift1['date']+pd.Timedelta('7 days'))
#平移第二次7天
link_traveling_time_shift_7_71_77_reshift2 = link_traveling_time_shift_7_71_77_reshift1.copy()
link_traveling_time_shift_7_71_77_reshift2['date'] = pd.DataFrame(link_traveling_time_shift_7_71_77_reshift2['date']+pd.Timedelta('7 days'))
#平移第三次7天
link_traveling_time_shift_7_71_77_reshift3 = link_traveling_time_shift_7_71_77_reshift2.copy()
link_traveling_time_shift_7_71_77_reshift3['date'] = pd.DataFrame(link_traveling_time_shift_7_71_77_reshift3['date']+pd.Timedelta('7 days'))
#平移第四次7天
link_traveling_time_shift_7_71_77_reshift4 = link_traveling_time_shift_7_71_77_reshift3.copy()
link_traveling_time_shift_7_71_77_reshift4['date'] = pd.DataFrame(link_traveling_time_shift_7_71_77_reshift4['date']+pd.Timedelta('7 days'))
#平移第五次7天
link_traveling_time_shift_7_71_77_reshift5 = link_traveling_time_shift_7_71_77_reshift4.copy()
link_traveling_time_shift_7_71_77_reshift5['date'] = pd.DataFrame(link_traveling_time_shift_7_71_77_reshift5['date']+pd.Timedelta('7 days'))

link_traveling_time_shift_7_prediction = pd.concat([link_traveling_time_shift_7_71_77, link_traveling_time_shift_7_71_77_reshift1, link_traveling_time_shift_7_71_77_reshift2,
                                                   link_traveling_time_shift_7_71_77_reshift3, link_traveling_time_shift_7_71_77_reshift4,
                                                   link_traveling_time_shift_7_71_77_reshift5])
link_traveling_time_shift_7_prediction.head()

,link_ID,date,time_interval_hour,time_interval_minutes,travel_time_7days_before
3794582,3377906280028510514,2017-07-01,00,00,5.0
3752937,3377906280028510514,2017-07-01,00,02,5.0
3780877,3377906280028510514,2017-07-01,00,04,5.0
3724452,3377906280028510514,2017-07-01,00,06,5.0
3775500,3377906280028510514,2017-07-01,00,08,4.1


In [110]:
prediction_table = pd.merge(prediction_table, link_traveling_time_shift_7_prediction, on=['link_ID', 'date', 'time_interval_hour', 'time_interval_minutes'], how='left')
len(prediction_table)

368280

In [111]:
link_traveling_time_shift_15_71_715 = link_traveling_time_shift_15[(link_traveling_time_shift_15['date']>='2017-07-01')&(link_traveling_time_shift_15['date']<='2017-07-15')]
#平移第一次15天
link_traveling_time_shift_15_71_715_reshift1 = link_traveling_time_shift_15_71_715.copy()
link_traveling_time_shift_15_71_715_reshift1['date'] = pd.DataFrame(link_traveling_time_shift_15_71_715_reshift1['date']+pd.Timedelta('15 days'))
#平移第二次15天
link_traveling_time_shift_15_71_715_reshift2 = link_traveling_time_shift_15_71_715_reshift1.copy()
link_traveling_time_shift_15_71_715_reshift2['date'] = pd.DataFrame(link_traveling_time_shift_15_71_715_reshift2['date']+pd.Timedelta('15 days'))

link_traveling_time_shift_15_prediction = pd.concat([link_traveling_time_shift_15_71_715, link_traveling_time_shift_15_71_715_reshift1,
                                                    link_traveling_time_shift_15_71_715_reshift2])
link_traveling_time_shift_15_prediction.head()

,link_ID,date,time_interval_hour,time_interval_minutes,travel_time_15days_before
4189847,3377906280028510514,2017-07-01,00,00,4.1
4205842,3377906280028510514,2017-07-01,00,02,3.8
4200012,3377906280028510514,2017-07-01,00,04,3.8
4198227,3377906280028510514,2017-07-01,00,06,3.8
4173337,3377906280028510514,2017-07-01,00,08,3.8


In [112]:
len(link_traveling_time_shift_15_prediction)

3780021

In [113]:
link_traveling_time_shift_15_prediction['date'].unique()

array(['2017-07-01T00:00:00.000000000', '2017-07-02T00:00:00.000000000',
       '2017-07-03T00:00:00.000000000', '2017-07-04T00:00:00.000000000',
       '2017-07-05T00:00:00.000000000', '2017-07-06T00:00:00.000000000',
       '2017-07-07T00:00:00.000000000', '2017-07-08T00:00:00.000000000',
       '2017-07-09T00:00:00.000000000', '2017-07-10T00:00:00.000000000',
       '2017-07-11T00:00:00.000000000', '2017-07-12T00:00:00.000000000',
       '2017-07-13T00:00:00.000000000', '2017-07-14T00:00:00.000000000',
       '2017-07-15T00:00:00.000000000', '2017-07-16T00:00:00.000000000',
       '2017-07-17T00:00:00.000000000', '2017-07-18T00:00:00.000000000',
       '2017-07-19T00:00:00.000000000', '2017-07-20T00:00:00.000000000',
       '2017-07-21T00:00:00.000000000', '2017-07-22T00:00:00.000000000',
       '2017-07-23T00:00:00.000000000', '2017-07-24T00:00:00.000000000',
       '2017-07-25T00:00:00.000000000', '2017-07-26T00:00:00.000000000',
       '2017-07-27T00:00:00.000000000', '2017-07-28

In [114]:
prediction_table = pd.merge(prediction_table, link_traveling_time_shift_15_prediction, on=['link_ID', 'date', 'time_interval_hour', 'time_interval_minutes'], how='left')
len(prediction_table)

368280

In [116]:
prediction_table.isnull().any()

link_ID                      False
date                         False
time_interval                False
travel_time                  False
time_interval_begin          False
time_interval_end            False
time_interval_hour           False
time_interval_minutes        False
interval_num                 False
mean_last_1hour              False
median_last_1hour            False
std_last_1hour               False
mode_last_1hour              False
max_last_1hour               False
min_last_1hour               False
mean_last_half_hour          False
median_last_half_hour        False
std_last_half_hour           False
mode_last_half_hour          False
max_last_half_hour           False
min_last_half_hour           False
time_stage1                  False
time_stage2                  False
time_stage3                  False
interval_num_1               False
interval_num_2               False
interval_num_3               False
interval_num_4               False
interval_num_5      

In [117]:
prediction_table = prediction_table.fillna(method = 'bfill')

In [118]:
prediction_table = pd.merge(prediction_table, temp_stats_all, on=['link_ID', 'time_interval_hour', 'time_interval_minutes'], how='left')

In [119]:
#将travel time 移到最后一列（这一步最后再做）
prediction_table['travel_time_tmp'] = prediction_table['travel_time']
del prediction_table['travel_time']
prediction_table['travel_time'] = prediction_table['travel_time_tmp']
del prediction_table['travel_time_tmp']

In [120]:
prediction_table.head()

,link_ID,date,time_interval,time_interval_begin,time_interval_end,time_interval_hour,time_interval_minutes,interval_num,mean_last_1hour,median_last_1hour,...,width_std,in_links_num_std,out_links_num_std,travel_time_7days_before,travel_time_15days_before,mean_all,median_all,std_all,mode_all,travel_time
0,3377906280028510514,2017-07-01,"[2017-07-01 08:00:00,2017-07-01 08:02:00)",2017-07-01 08:00:00,2017-07-01 08:02:00,08,00,0.0,5.742857,5.4,...,0.0,0.25,0.25,18.8,4.4,6.195238,5.40,2.018808,5.0,0
1,3377906280028510514,2017-07-01,"[2017-07-01 08:02:00,2017-07-01 08:04:00)",2017-07-01 08:02:00,2017-07-01 08:04:00,08,02,1.0,5.742857,5.4,...,0.0,0.25,0.25,18.8,4.4,6.628205,5.70,2.902082,4.8,0
2,3377906280028510514,2017-07-01,"[2017-07-01 08:04:00,2017-07-01 08:06:00)",2017-07-01 08:04:00,2017-07-01 08:06:00,08,04,2.0,5.742857,5.4,...,0.0,0.25,0.25,13.6,4.4,5.944186,5.40,2.180575,4.9,0
3,3377906280028510514,2017-07-01,"[2017-07-01 08:06:00,2017-07-01 08:08:00)",2017-07-01 08:06:00,2017-07-01 08:08:00,08,06,3.0,5.742857,5.4,...,0.0,0.25,0.25,13.2,4.4,6.100000,5.70,2.270162,6.0,0
4,3377906280028510514,2017-07-01,"[2017-07-01 08:08:00,2017-07-01 08:10:00)",2017-07-01 08:08:00,2017-07-01 08:10:00,08,08,4.0,5.742857,5.4,...,0.0,0.25,0.25,15.3,4.4,6.540476,5.55,3.196802,4.8,0


In [121]:
# 为了保证pandas读取link_ID是str类型
travel_time_train['link_ID'] = travel_time_train['link_ID'].astype(str) + '_'
prediction_table['link_ID'] = prediction_table['link_ID'].astype(str) + '_'

In [124]:
travel_time_train.to_csv('./preprocessed_data/travel_time_train.txt', sep=';', index=False)
prediction_table.to_csv('./preprocessed_data/prediction_table.txt', sep=';', index=False)

In [122]:
len(travel_time_train)

808159

In [123]:
len(prediction_table)

368280